# SPICE data analysis demo

From within a new environment:
```sh
python -m venv venv
. venv/bin/activate
python -m pip install -r requirements.txt
```
This will also install the `jupyter` package. Then, still within the environment, install it for Jupyter with
```
ipython kernel install --user --name=spice-demo-isro
```
and run
```
jupyter notebook
```
then open this notebook and run it with the kernel `spice-demo-isro`.

## Access to catalogs

A catalog of SPICE observations is provided with each data release. The latest release and catalog URL are determined automatically. The catalog is downloaded and cached using astropy.

In [ ]:
from sospice import Catalog

To access a catalog as a local file (not demonstrated here):
```python
catalog = Catalog(filename)
```

To access the catalog from the latest data release:

In [ ]:
catalog = Catalog(release_tag="latest")
print(f"Catalog length: {len(catalog)}")

The catalog in the cache can be update by adding the `update_cache=True` argument.

List all columns of the catalog:

In [ ]:
", ".join(sorted(catalog.columns))

## Queries in the catalog

Queries can be done on the catalog using `Catalog.find_files()`, which offers different options for different types of queries.

In [ ]:
# Find file closest to some date
catalog.find_files(closest_to_date="2022-04-02T13")

The `Catalog` object is based on `pandas.DataFrame()`, so many `pandas` methods can be used (and dates are parsed by `pandas`). The object returned by `.find_files()` is itself a `Catalog`, so queries can be made step by step (or chained).

In [ ]:
# Find files in some date range
files1 = catalog.find_files(date_min="2022-04-01", date_max="10 Apr 2022 12:00")
files1

In [ ]:
# Get all available SOOP names in this sub-catalog (corresponding to the time range above)
files1.SOOPNAME.unique()

Keyword values equalities can also be queried directly by using the keywords as argument to `find_files()` (lower case can be used):

In [ ]:
# All science files for a SOOP, in the above time range
files2 = files1.find_files(
    level="L2",
    soopname="R_SMALL_MRES_MCAD_AR-Long-Term",
    purpose='Science'
)
files2

In [ ]:
# Only display some columns for the first occurrence of each study in the SOOP.
# This allows retrieving parameters that should be common to all occurrences of each study, such as NAXISi, XPOSURE...
# Of course parameters such as DATE-BEG change for each observation, so only the value for the first observation of each study is displayed here.
selected_columns = ["DATE-BEG", "NAXIS1", "NAXIS2", "NAXIS3", "NAXIS4", "XPOSURE", "STUDY"]
files2[selected_columns].groupby("STUDY").first()

Queries involving equalities or inequalities of values of the headers (string or number values, but not dates) can be done with the `query` keyword. This uses `pandas.DataFrame.query()`. Comparisons can be chained to be used for intervals, as [in standard Python](https://docs.python.org/3/reference/expressions.html#comparisons).

In [ ]:
selected_columns += ["CRVAL1", "CRVAL2"]
files2.find_files(query="-2000 < CRVAL1 < -1000 & CRVAL2 > 600")[selected_columns]

In [ ]:
# Rather get all observations made with studies designed for "composition" (abundances) diagnostics (i.e. having "COMPO" in their names)
files3 = files2[files2.STUDY.str.contains("COMPO")]
files3[selected_columns]

`files3` was a `DataFrame`, convert it back to a `Catalog` (this shouldn't be needed anymore [in the future](https://github.com/solo-spice/sospice/issues/42#issuecomment-1923525165)). 

In [ ]:
files3 = Catalog(data_frame=files3)

## Plot fields of view

When we have a subset of the SPICE catalog, we can plot all fields of view for the different SPICE observations in the sub-catalog.

Here the EUI/FSI image in the background is selected automatically, from the midpoint of the time range of SPICE observations in the sub-catalog.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib ipympl
from sospice import plot_fovs_with_background
plot_fovs_with_background(files3, 'EUI/FSI')

## Using a file's metadata

Each row in the catalog correspond to a SPICE file's metadata.

In [ ]:
import astropy.units as u
from sospice import FileMetadata

In [ ]:
metadata = FileMetadata(files3.iloc[0])

### Getting the wavelength ranges

From data release 3.0, the WAVECOV keyword contains a list of wavelength ranges for the windows, this can be accessed by:

In [ ]:
wavelength_ranges = metadata.get_wavelengths()

This is the union of several intervals, described by a `portion` object, making it easy to check whether a wavelength is included:

In [ ]:
print(wavelength_ranges)
77 * u.nm in wavelength_ranges

## Access a file

In [ ]:
from astropy.io import fits

The file can be put in the `astropy` cache (and then the cache file can be used, downloaded to a directory tree at a specific location, or be used directly from its URL.

Files are downloaded from SOAR if no release and no other base URL is provided.
If a release is provided, it must be consistent with the one corresponding to the catalog (a file from some release is likely not to be present in another release).

### From URL

In [ ]:
url = metadata.get_file_url(release="latest")
url

In [ ]:
hdulist = fits.open(url)
hdulist.info()

In [ ]:
hdu_ne8 = hdulist[3]

In [ ]:
hdu_ne8.data.shape

In [ ]:
ix = 67
data = hdu_ne8.data[0, :, :, ix]

In [ ]:
# detector image (λ, y), at some x
plt.figure()
plt.imshow(data.T, vmin=0, vmax=20, aspect=1/10, origin="lower")
plt.colorbar()
plt.title("Ne VIII")
plt.show()

The bright line at the top (in this plot and in other plots below) corresponds to the bright "dumbbell", a wider part of the slit, meant to help co-alignement with imaging data.

### Downloading to a local SPICE file tree

In [ ]:
directory = '/tmp/spice-files'
metadata.download_file(directory, release="latest")

This can be easily applied to several files in the catalog:

In [ ]:
from parfive import Downloader
downloader = Downloader()
files3.apply(
  lambda row: FileMetadata(row).download_file(directory, release="latest", downloader=downloader),
  axis=1
)
downloader.download()

This function returns the downloaded file names, that can then be used later in the script.

As `parfive` does not redownload already-downloaded files (unless requested), this is somewhat equivalent to using the cache, with more explicit file names but less protection against network errors.

## Displaying data with axes

The (λ, y) image above has been obtained by simply displaying a HDU data array from the FITS file after reading it with `astropy.io.fits.open()`. The drawback is that information about axes (WCS coordinates) has been lost.

### With sunraster

`sunraster` (generic slit spectrograph data analysis package) contains a SPICE L2 FITS file reader. The resulting object is derived from [NDCube](https://docs.sunpy.org/projects/ndcube/), which is convenient to manipulate data cubes with their axes. This reader will [soon be available from sospice](https://github.com/solo-spice/sospice/pull/56).


In [ ]:
from sunraster.instr.spice import read_spice_l2_fits
raster = read_spice_l2_fits(url)
raster

In [ ]:
window_ne8 = raster['Ne VIII 770 - Peak']
window_ne8

In [ ]:
# Select a value normalization for the next plot
from astropy.visualization import SqrtStretch, AsymmetricPercentileInterval, ImageNormalize
norm = ImageNormalize(window_ne8.data,
                      interval=AsymmetricPercentileInterval(1, 99.9),
                      stretch=SqrtStretch()
                     )

In [ ]:
# Display the window as an "animation": 2D (x, y) map where λ can be chosen from a slider.
window_ne8.plot(aspect="auto", norm=norm)
plt.show()
# Nothing is displayed at first, this is because the datacube is displayed at the minimal wavelength; then use the wavelength slider to select other wavelengths.

The aspect ratio is too wide, but surely this can be fixed.

### With `sunpy.Map`

An alternative is to make a sunpy Map out of the data and metadata.

In [ ]:
# select some central wavelength, giving a 2D (x, y) map
window_ne8_peak = window_ne8[0, 25, :, :]

In [ ]:
from sunpy.map import Map
m_spice = Map((window_ne8_peak.data, window_ne8_peak.meta))
m_spice.plot_settings['cmap'] = plt.get_cmap('viridis')
m_spice.plot_settings['norm'] = norm

plt.figure()
m_spice.plot(norm=norm, aspect=1/4)  # 1/4 because raster step is 4", about 4 times the vertical pixel size
plt.colorbar()
plt.show()

In [ ]:
# Display the spectrum at some pixel
window_ne8[0, :, 674, 60].plot()
plt.show()

## Compute uncertainties due to noises

This uses instrument modelling, with some calibration parameters.

In [ ]:
from sospice import spice_error

In [ ]:
av_noise_contribution, sigma = spice_error(hdu_ne8)

In [ ]:
av_noise_contribution

In [ ]:
sigma.keys()

In [ ]:
# Noise from the dark current (a scalar)
sigma['Dark']

In [ ]:
# Read noise (as scalar)
sigma['Read']

In [ ]:
# Total uncertainty due to the different noise sources (a data cube; here we select a place at some x)
uncertainty_data = sigma["Total"][0, :, :, ix]

In [ ]:
plt.figure()
plt.imshow(uncertainty_data.T.value, vmin=0, vmax=.7, aspect=1/10, origin="lower")
plt.colorbar()
plt.title("Ne VIII uncertainty")
plt.show()

## Other resources

* [SPICE data analysis user's manual](https://spice-wiki.ias.u-psud.fr/doku.php/data:data_analysis_manual)
* [SPICE data quicklook](https://git.ias.u-psud.fr/spice/data_quicklook): SPICE-IT-UP (L2 FITS file quicklook tool)
* [SPICE jitter correction](https://github.com/gpelouze/spice_jitter_correction): uses WCSDVAR the remap the data cubes to correct for pointing jitter. This should also soon be included in sospice.
* Parallel fitting has been implemented in `astropy.modelling` (astropy ≥ 7.0) ([tutorial](https://github.com/aperiosoftware/parallel-modeling-examples/blob/main/spice/Final_Tutorial.ipynb) by Stuart Mumford).

* Line fitting is also possible with the [SAFFRON package](https://github.com/slimguat/saffron-spice)